# Generating Borders

When geographic data shares common borders, a lot of data processing and mapping is duplicated. Topologies can reduce that by referring to the relationships between items rather than treating each area as a separate entity.

## Creating Borders

This is the procedure followed for generating the "borders" from simple locations.
In this instance it's for PNG but the same procedure should work for any geographic data.
First create a new "topology"

```sql
SELECT topology.CreateTopology('ma_topo', 3857, 15);  -- should return 1 if this is the first topology to be created
CREATE TABLE public.simple_locations_topology(id serial, area_id int);
SELECT topology.AddTopoGeometryColumn('ma_topo', 'public', 'simple_locations_topology', 'topo', 'LINESTRING');
```

Process existing georaphic data to topology. This can take a long time. If you encounter exceptions, check that your geometry
is valid and try different simplification

```sql
insert into simple_locations_topology(area_id, topo)  SELECT id, topology.toTopoGeom(st_transform(ST_ExteriorRing((st_dump(geom)).geom), 3857), 'ma_topo', 1, 15)
from simple_locations_area sla;

drop table if exists simple_locations_areaboundary;
create table if not exists simple_locations_areaboundary(geom geometry(linestring, 3857), edge_id int, area_ids int[], area_types text[]);
truncate simple_locations_areaboundary;
insert into simple_locations_areaboundary(geom, edge_id, area_ids, area_types)
select 
	e.geom, e.edge_id, edge_area_kind.area_ids, edge_area_kind.area_types
	from ma_topo.edge e,
	(
	SELECT e.edge_id, array_agg( distinct sla2.id) area_ids, array_agg( distinct sla3.slug) area_types
	FROM ma_topo.edge e,
			ma_topo.relation rel,
	     	simple_locations_topology sla,
	     	simple_locations_area sla2,
	     	simple_locations_areatype sla3
	WHERE e.edge_id = rel.element_id
	  AND rel.topogeo_id = (sla.topo).id
	  and sla.area_id = sla2.id
	  and sla2.kind_id = sla3.id
	  and rel.layer_id = 1
	  group by e.edge_id
	) edge_area_kind
	where e.edge_id = edge_area_kind.edge_id;

insert into simple_locations_border(geom) select geom from simple_locations_areaboundary;

insert into simple_locations_border_area(border_id, area_id)
select b.id, unnest(ab.area_ids) from simple_locations_areaboundary ab, simple_locations_border b where b.geom = ab.geom
```

An example table from the above, lines which are regional borders:

```sql
create table region_borders as (
SELECT DISTINCT "simple_locations_border"."geom"
FROM "simple_locations_border" 
INNER JOIN "simple_locations_border_area" 
ON ("simple_locations_border"."id" = "simple_locations_border_area"."border_id")
INNER JOIN "simple_locations_area" ON ("simple_locations_border_area"."area_id" = "simple_locations_area"."id") 
INNER JOIN "simple_locations_areatype" ON ("simple_locations_area"."kind_id" = "simple_locations_areatype"."id") 
WHERE "simple_locations_areatype"."slug" = 'region'
);
```